In [291]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import statsmodels.api as sm
import warnings
import dask.dataframe as dd
warnings.filterwarnings('ignore')
from io import StringIO
from pypfopt import black_litterman, risk_models, BlackLittermanModel
from sklearn.covariance import LedoitWolf
import cvxpy as cp
from numpy.linalg import pinv


In [ ]:
# Load the returns data
df = pd.read_excel(
    'Data/DS_RI_T_USD_M.xlsx',
    header=None,
)

# Transpose the DataFrame
df = df.T

# Set the second row (index 1) as the column headers
df.columns = df.iloc[0]
column_names = df.iloc[1].values
print(column_names)

# Remove the first two rows as they are now redundant
df = df.drop([0, 1])

# Rename the first column to 'Date' and set it as the index
df = df.rename(columns={df.columns[0]: "Date"}).set_index("Date")

# Convert all entries to floats for uniformity and handling
df = df.astype(float)

# Initialize a set to keep track of dropped stocks
dropped_stocks = set()

# 1. Remove stocks with initial zero prices
initial_zeros = df.iloc[0] == 0
dropped_stocks.update(df.columns[initial_zeros])
print(f"Initial zero : {df.columns[initial_zeros]}")
df = df.loc[:, ~initial_zeros]

# 2. Remove stocks that ever drop to zero
ever_zeros = (df == 0).any()
dropped_stocks.update(df.columns[ever_zeros])
print(f"Ever zero : {df.columns[ever_zeros]}")
df = df.loc[:, ~ever_zeros]

# 3. Remove stocks that do not recover after dropping to zero
max_prior = df.cummax()
recovered = ((df / max_prior.shift()) > 0.1).any()
non_recovered = df.columns[~recovered]
dropped_stocks.update(non_recovered)
print(f"Non recovered : {non_recovered}")
df = df.loc[:, recovered]

# # Filter based on sector information
# static_file = pd.read_excel(
#     r"C:\Users\marko\OneDrive\Bureau\Marko_documents\Etudes\Master_2ème\1er_semestre\Quantitative Risk and Asset Management 2\Projet_PortfolioOptimization\Data\Static.xlsx"
# )
# sectors = ["Energy", "Materials", "Utilities", "Industrials"]
# companies = static_file[static_file["GICSSectorName"].isin(sectors)]
# isin_list = companies["ISIN"].tolist()

# # Identify stocks that are not in the highly polluting sectors
# non_polluting_stocks = set(df.columns) - set(isin_list)
# dropped_stocks.update(non_polluting_stocks)

# df = df[df.columns.intersection(isin_list)]


# # Reset column names to the original names after modifications
# df.columns = column_names[
#     1 : len(df.columns) + 1
# ]  # Skip the first name since it corresponds to the Date column

# Proceed with any further data processing, such as calculating returns
monthly_returns = df.pct_change()
monthly_returns = monthly_returns.drop(monthly_returns.index[0])

# Handling NaN and infinite values
monthly_returns.replace([np.inf, -np.inf], np.nan, inplace=True)
monthly_returns.interpolate(method="linear", axis=0, inplace=True)
monthly_returns.fillna(method="ffill", axis=0, inplace=True)
monthly_returns.fillna(method="bfill", axis=0, inplace=True)

# Display results
print("Remaining NaN values in monthly returns:", monthly_returns.isnull().sum().sum())
# df.to_csv("Cleaned_df.csv", index=True)
# monthly_returns.to_csv("Cleaned_df_returns.csv", index=True)

In [ ]:

# Load Pickle (if possible)
large_data = pd.read_pickle('df_withcorona_clean_1_with_proba_opti_and_hour.pkl')

# Save in chunks as Parquet
chunk_size = 1_000_000
for i in range(0, len(large_data), chunk_size):
    chunk = large_data.iloc[i:i+chunk_size]
    chunk.to_parquet(f"large_data_chunk_1_{i}.parquet")

In [2]:
# Use the same file pattern to read all parquet files into a Dask DataFrame
file_pattern = 'large_data_chunk_*.parquet'
ddf = dd.read_parquet(file_pattern, engine='pyarrow')

In [3]:
# Dask can handle datetime conversion efficiently
ddf['date'] = dd.to_datetime(ddf['date'])
ddf['date_only'] = ddf['date'].dt.date


In [4]:
print(len(ddf))

90900636


In [5]:
# Group by ticker and date, compute sum and count
grouped_ddf = ddf.groupby(['ticker', 'date_only'])['sent_merged'].agg(['sum', 'count'])

# Compute the average sentiment
grouped_ddf['Average_Sentiment'] = grouped_ddf['sum'] / grouped_ddf['count']

# Reset index to turn indices into columns
grouped_ddf = grouped_ddf.reset_index()

# To compute and get the result, you need to call compute()
aggregated_data = grouped_ddf.compute()

# Rename columns for clarity
aggregated_data.columns = ['Ticker', 'Date', 'Sentiment_Sum', 'Sentiment_Count', 'Average_Sentiment']

# Display the result
print(aggregated_data.head())


  Ticker        Date  Sentiment_Sum  Sentiment_Count  Average_Sentiment
0   STEM  2016-08-16           4338             6205           0.699114
1   LABD  2016-08-16             32               81           0.395062
2   TWLO  2016-08-16            398              702           0.566952
3   AUPH  2016-08-16           1112             1445           0.769550
4   JBLU  2016-08-16             15               27           0.555556


In [6]:
aggregated_data.to_csv('average_daily_sentiment_per_ticker.csv', index=False)

In [7]:
# Read the average daily sentiment per ticker
sentiment_df = pd.read_csv('average_daily_sentiment_per_ticker.csv')

# Ensure the 'Date' column is in datetime format
sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'])


In [8]:
# Read the ticker to sector mapping from the Excel file
ticker_sector_df = pd.read_csv("ticker_sector_corrected_dollar_format.csv", delimiter='$')

# Display the first few rows (optional)
print(ticker_sector_df.head())


  Ticker                   Company Name                  Sector
0   AAPL                     Apple Inc.  Information Technology
1     AA              Alcoa Corporation               Materials
2   AAOI  Applied Optoelectronics, Inc.  Information Technology
3    AAP       Advance Auto Parts, Inc.  Consumer Discretionary
4    DBV                        Unknown                 Unknown


In [9]:
# Merge the sentiment DataFrame with the sector DataFrame
merged_df = pd.merge(sentiment_df, ticker_sector_df[['Ticker', 'Sector']], on='Ticker', how='left')

# Check for any missing sectors
missing_sectors = merged_df[merged_df['Sector'].isnull()]['Ticker'].unique()
if len(missing_sectors) > 0:
    print("Tickers with missing sector information:", missing_sectors)


Tickers with missing sector information: ['STEM' 'LABD' 'TWLO' ... 'ROME' 'PIQ' 'TWON']


In [10]:
# Option 1: Remove records with missing sectors
merged_df = merged_df.dropna(subset=['Sector'])

# Option 2: Fill missing sectors with 'Unknown'
# merged_df['Sector'] = merged_df['Sector'].fillna('Unknown')


In [11]:
# Group by Sector and Date to compute average sentiment
sector_sentiment = merged_df.groupby(['Sector', 'Date']).agg({
    'Sentiment_Sum': 'sum',
    'Sentiment_Count': 'sum'
}).reset_index()

# Calculate the average sentiment
sector_sentiment['Average_Sentiment'] = sector_sentiment['Sentiment_Sum'] / sector_sentiment['Sentiment_Count']

# Display the result
print(sector_sentiment.head())


                   Sector       Date  Sentiment_Sum  Sentiment_Count  \
0  Communication Services 2009-07-13              3                3   
1  Communication Services 2009-08-20              1                1   
2  Communication Services 2009-08-21              1                1   
3  Communication Services 2009-08-31              5                5   
4  Communication Services 2009-09-01              1                4   

   Average_Sentiment  
0               1.00  
1               1.00  
2               1.00  
3               1.00  
4               0.25  


In [12]:
# Save the average daily sentiment per sector to a CSV file
sector_sentiment.to_csv('average_daily_sentiment_per_sector.csv', index=False)


In [21]:
sentiment_df = pd.read_csv('average_daily_sentiment_per_sector.csv')
sentiment_df

,Sector,Date,Sentiment_Sum,Sentiment_Count,Average_Sentiment
0,Communication Services,2009-07-13,3,3,1.000000
1,Communication Services,2009-08-20,1,1,1.000000
2,Communication Services,2009-08-21,1,1,1.000000
3,Communication Services,2009-08-31,5,5,1.000000
4,Communication Services,2009-09-01,1,4,0.250000
...,...,...,...,...,...
47243,Unknown,2020-03-19,455,1170,0.388889
47244,Unknown,2020-03-20,420,921,0.456026
47245,Unknown,2020-03-21,159,297,0.535354
47246,Unknown,2020-03-22,145,332,0.436747


In [22]:
# Ensure the Date column is a datetime object
sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'])

# Create a new column for the month
sentiment_df['Month'] = sentiment_df['Date'].dt.to_period('M')

# Group by Sector and Month, and aggregate
monthly_df = sentiment_df.groupby(['Sector', 'Month']).agg({
    'Sentiment_Sum': 'sum',  # Sum of Sentiment_Sum
    'Sentiment_Count': 'sum',  # Sum of Sentiment_Count
    'Average_Sentiment': 'mean'  # Average of Average_Sentiment
}).reset_index()

# Convert 'Month' back to datetime for easier use if needed
monthly_df['Month'] = monthly_df['Month'].dt.to_timestamp()

# Rename the Month column to Date for consistency
monthly_df.rename(columns={'Month': 'Date'}, inplace=True)

monthly_df

,Sector,Date,Sentiment_Sum,Sentiment_Count,Average_Sentiment
0,Communication Services,2009-07-01,3,3,1.000000
1,Communication Services,2009-08-01,7,7,1.000000
2,Communication Services,2009-09-01,117,146,0.833594
3,Communication Services,2009-10-01,97,138,0.763863
4,Communication Services,2009-11-01,27,192,0.405869
...,...,...,...,...,...
1667,Unknown,2019-11-01,11171,17143,0.669213
1668,Unknown,2019-12-01,12920,24939,0.594906
1669,Unknown,2020-01-01,14577,23702,0.643206
1670,Unknown,2020-02-01,12507,20007,0.618926


In [24]:
monthly_df = monthly_df.where(monthly_df["Sentiment_Count"] >= 50)
monthly_df = monthly_df.dropna()
monthly_df = monthly_df[monthly_df["Sector"] != "Unknown"]
sentiment_df = monthly_df
sentiment_df

,Sector,Date,Sentiment_Sum,Sentiment_Count,Average_Sentiment
2,Communication Services,2009-09-01,117.0,146.0,0.833594
3,Communication Services,2009-10-01,97.0,138.0,0.763863
4,Communication Services,2009-11-01,27.0,192.0,0.405869
5,Communication Services,2009-12-01,74.0,109.0,0.671356
6,Communication Services,2010-01-01,199.0,269.0,0.637201
...,...,...,...,...,...
1538,Real Estate,2019-11-01,628.0,935.0,0.675777
1539,Real Estate,2019-12-01,615.0,888.0,0.709744
1540,Real Estate,2020-01-01,668.0,1066.0,0.673506
1541,Real Estate,2020-02-01,1037.0,1705.0,0.626191


In [25]:
def initialize_data():
    static_df = pd.read_excel("Data/Static.xlsx")

    df = pd.read_csv("Cleaned_df.csv", index_col="Date")
    df.index = pd.to_datetime(df.index)
    return static_df, df

static_df, df = initialize_data()

In [26]:
# Map assets to companies
asset_names = static_df['Company'].tolist()
if len(asset_names) >= len(df.columns):
    df.columns = asset_names[:len(df.columns)]
else:
    df.columns = [f'Asset_{i}' for i in range(1, len(df.columns)+1)]

In [167]:
# Step 3: Calculate historical returns and covariance matrix
returns = df.pct_change().dropna()

# Calculate the sample covariance matrix of returns
cov_matrix = risk_models.CovarianceShrinkage(df, frequency=12).ledoit_wolf()

# Regularize covariance matrix (after Ledoit-Wolf shrinkage)
epsilon = 1e-4  # Small regularization factor
cov_matrix += epsilon * np.eye(cov_matrix.shape[0])


In [168]:
returns

,Schlumberger Limited,Aluar Aluminio Argentino S.A.I.C.,Banco BBVA Argentina S.A.,Ternium Argentina S.A.,Flughafen Wien Aktiengesellschaft,Erste Group Bank AG,OMV Aktiengesellschaft,VERBUND AG,Wienerberger AG,Vienna Insurance Group AG,...,Harmony Gold Mining Company Limited,Gold Fields Limited,Discovery Limited,Barloworld Limited,Truworths International Limited,MTN Group Limited,African Rainbow Minerals Limited,Reunert Limited,Woolworths Holdings Limited,FirstRand Limited
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-29,0.213043,0.050002,0.178923,0.039543,-0.015488,0.029818,-0.181937,0.076230,-0.073916,-0.032720,...,-0.088413,-0.037800,-0.326248,0.028928,-0.053481,-0.084315,-0.062844,-0.181598,0.018379,-0.023150
2000-03-31,0.035756,-0.023552,-0.105072,-0.002113,0.031138,0.028238,-0.023027,-0.081528,0.112531,-0.005999,...,0.079289,-0.067887,-0.044066,-0.176088,-0.126866,0.073915,-0.054382,-0.038462,-0.142989,-0.104274
2000-04-30,0.000814,-0.016173,-0.161842,-0.071188,-0.134598,-0.043483,0.076523,-0.106429,0.041304,-0.021831,...,-0.106082,-0.034586,-0.172342,-0.129131,-0.035409,-0.208833,-0.030503,-0.224615,-0.095572,0.003496
2000-05-31,-0.039184,-0.016727,0.005304,-0.341421,0.085088,0.017996,0.064640,0.074843,0.075991,0.049232,...,-0.007204,-0.123979,0.012025,-0.026810,-0.003797,-0.151722,0.034269,0.063492,-0.215522,-0.188041
2000-06-30,0.017198,-0.081874,0.139652,0.070129,0.072038,0.042918,0.021630,0.000964,0.010601,0.039414,...,0.067594,0.068384,-0.239525,0.130112,-0.052097,-0.028701,0.035264,0.287313,0.131659,0.157602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.030245,-0.033996,0.081313,0.039595,-0.010844,-0.102476,-0.040985,-0.125302,0.025707,0.041194,...,-0.063886,-0.017240,-0.239233,-0.125554,-0.081044,0.025119,0.023407,-0.106945,-0.117647,-0.054408
2022-09-30,-0.054569,-0.112147,-0.001978,-0.115037,-0.021276,-0.018766,-0.096105,-0.106440,-0.140093,-0.139022,...,-0.077244,-0.117696,-0.172806,-0.152027,-0.122378,-0.041751,-0.024755,-0.077264,-0.133333,-0.262928
2022-10-31,0.449301,-0.004103,-0.067049,0.017780,0.007311,0.112197,0.259085,-0.085763,0.133224,0.093316,...,0.165741,0.057276,0.037358,-0.043327,0.130410,-0.014151,0.105902,0.074825,0.000000,-0.005929


In [169]:
# Map each asset to its sector
asset_sector_map = static_df.set_index('Company')['GICSSectorName'].to_dict()

# Create a DataFrame with asset names and their sectors
asset_sector_df = pd.DataFrame({'Asset': df.columns})
asset_sector_df['GICSSectorName'] = asset_sector_df['Asset'].map(asset_sector_map)

In [170]:
asset_sector_df

,Asset,GICSSectorName
0,Schlumberger Limited,Energy
1,Aluar Aluminio Argentino S.A.I.C.,Materials
2,Banco BBVA Argentina S.A.,Financials
3,Ternium Argentina S.A.,Materials
4,Flughafen Wien Aktiengesellschaft,Industrials
...,...,...
2028,MTN Group Limited,Communication Services
2029,African Rainbow Minerals Limited,Materials
2030,Reunert Limited,Information Technology
2031,Woolworths Holdings Limited,Consumer Discretionary


In [171]:
# Calculate the average sentiment per sector over the entire period
sector_sentiment = sentiment_df.groupby('Sector')['Average_Sentiment'].mean()

In [172]:
# Create the views (Q) based on average sentiment per sector
Q = sector_sentiment.values  # Expected returns adjustments per sector

# Create the P matrix
num_assets = len(df.columns)
num_sectors = len(sector_sentiment)
P = pd.DataFrame(0, index=sector_sentiment.index, columns=df.columns)

In [173]:
Q

array([0.53744988, 0.55552397, 0.71969462, 0.70576312, 0.41874066,
       0.63906432, 0.59633685, 0.74893379, 0.5803541 , 0.63909876,
       0.66972478, 0.6611545 ])

In [174]:
# Map each asset to its sector and construct P matrix
for sector in sector_sentiment.index:
    assets_in_sector = asset_sector_df[asset_sector_df['GICSSectorName'] == sector]['Asset']
    n_assets_in_sector = len(assets_in_sector)
    if n_assets_in_sector > 0:
        P.loc[sector, assets_in_sector] = 1 / n_assets_in_sector  # Equal weights

# Convert P to numpy array
P = P.values


In [175]:
P

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.003367  ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00518135, 0.        ,
        0.        ],
       [0.        , 0.00431034, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [176]:
# Set tau (scaling factor)
tau = 0.05  # Adjust as necessary

# Calculate omega (uncertainty matrix)
omega = np.diag(np.diag(tau * P @ cov_matrix.values @ P.T))

In [177]:
# Set initial weights x0 (assumed market cap weights or equal weights)
x0 = np.array([1/num_assets] * num_assets)

In [178]:
# Compute initial portfolio return and volatility
rf = 0.0  # Risk-free rate, adjust if necessary
initial_returns = returns.mean()
initial_portfolio_return = np.dot(x0, initial_returns)
initial_portfolio_vol = np.sqrt(np.dot(x0.T, np.dot(cov_matrix, x0)))
initial_portfolio_sharpe = (initial_portfolio_return - rf) / initial_portfolio_vol

# Compute implied risk aversion parameter (delta)
delta = initial_portfolio_sharpe / initial_portfolio_vol

# Compute the implied equilibrium returns (pi)
pi = delta * cov_matrix.dot(x0)

In [179]:
gamma = tau * cov_matrix
gamma

,Schlumberger Limited,Aluar Aluminio Argentino S.A.I.C.,Banco BBVA Argentina S.A.,Ternium Argentina S.A.,Flughafen Wien Aktiengesellschaft,Erste Group Bank AG,OMV Aktiengesellschaft,VERBUND AG,Wienerberger AG,Vienna Insurance Group AG,...,Harmony Gold Mining Company Limited,Gold Fields Limited,Discovery Limited,Barloworld Limited,Truworths International Limited,MTN Group Limited,African Rainbow Minerals Limited,Reunert Limited,Woolworths Holdings Limited,FirstRand Limited
Schlumberger Limited,0.006891,0.002124,0.003653,0.002508,0.002015,0.003092,0.003324,0.001707,0.002584,0.002080,...,0.002633,0.002069,0.003901,0.002012,0.001918,0.001803,0.002684,0.003422,0.002097,0.003620
Aluar Aluminio Argentino S.A.I.C.,0.002124,0.009874,0.006163,0.007343,0.001079,0.002008,0.001181,0.001710,0.002134,0.001304,...,0.001138,0.001184,0.001443,0.000457,0.001108,0.001125,0.001319,0.001372,0.001708,0.001757
Banco BBVA Argentina S.A.,0.003653,0.006163,0.015136,0.007939,0.002562,0.003568,0.002418,0.002592,0.002524,0.002654,...,0.003508,0.002637,0.003011,0.002159,0.002643,0.003095,0.003919,0.003083,0.003627,0.003546
Ternium Argentina S.A.,0.002508,0.007343,0.007939,0.012819,0.001263,0.002022,0.001559,0.001981,0.001598,0.001543,...,0.001471,0.001370,0.002409,0.000403,0.000918,0.001832,0.001520,0.001431,0.002227,0.001538
Flughafen Wien Aktiengesellschaft,0.002015,0.001079,0.002562,0.001263,0.004486,0.002964,0.002236,0.002109,0.002319,0.002118,...,0.001934,0.001669,0.003273,0.001863,0.001505,0.001535,0.001889,0.002367,0.001411,0.002807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTN Group Limited,0.001803,0.001125,0.003095,0.001832,0.001535,0.002990,0.002071,0.001638,0.002282,0.001982,...,0.002928,0.002632,0.003942,0.002610,0.002617,0.007188,0.002845,0.002755,0.003300,0.003068
African Rainbow Minerals Limited,0.002684,0.001319,0.003919,0.001520,0.001889,0.003358,0.002342,0.002231,0.002309,0.002022,...,0.005016,0.003734,0.004180,0.003736,0.003899,0.002845,0.006918,0.003340,0.003904,0.004907
Reunert Limited,0.003422,0.001372,0.003083,0.001431,0.002367,0.003620,0.002976,0.002562,0.003542,0.002176,...,0.003406,0.002849,0.005125,0.002964,0.002447,0.002755,0.003340,0.011371,0.003950,0.004036
Woolworths Holdings Limited,0.002097,0.001708,0.003627,0.002227,0.001411,0.003104,0.002187,0.001433,0.002634,0.002043,...,0.003665,0.003135,0.006168,0.005837,0.003401,0.003300,0.003904,0.003950,0.016035,0.004484


In [216]:
M = P.dot(gamma).dot(P.T) + omega

# Regularize covariance matrix (after Ledoit-Wolf shrinkage)
epsilon = 1e-25  # Small regularization factor
M += epsilon * np.eye(M.shape[0])

In [217]:
M_inverse = np.linalg.inv(M)

In [218]:
mu_bar = pi + gamma.dot(P.T).dot(M_inverse).dot(Q - P.dot(pi))
mu_bar

Schlumberger Limited                 0.781975
Aluar Aluminio Argentino S.A.I.C.    0.559009
Banco BBVA Argentina S.A.            0.836741
Ternium Argentina S.A.               0.537547
Flughafen Wien Aktiengesellschaft    0.608685
                                       ...   
MTN Group Limited                    0.617276
African Rainbow Minerals Limited     0.729807
Reunert Limited                      0.916152
Woolworths Holdings Limited          0.714852
FirstRand Limited                    0.923663
Length: 2033, dtype: float64

In [219]:
# Save the expected returns to a CSV file
expected_returns_df = pd.Series(mu_bar, index=df.columns, name='ExpectedReturn')
expected_returns_df.to_csv('expected_returns_conditional.csv', header=True)

print("Posterior expected returns saved to 'expected_returns_conditional.csv'")

Posterior expected returns saved to 'expected_returns_conditional.csv'


In [271]:

assets = len(returns)

# Prepare a result dictionary
result = {
    "weights": None,
    "mean_returns": None,
    "cov_matrix": cov_matrix,
    "status": None,
}

# Define variables
w = cp.Variable(num_assets)

# Objective function
portfolio_variance = cp.quad_form(w, cov_matrix, assume_PSD=True)

# Initialize constraints list
constraints = []


constraints.append(w >= -1)
constraints.append(w <= 1)
constraints.append(cp.sum(w) == 1)
constraints.append(cp.sum(cp.abs(w)) <= 5)

# Maximize Utility Function: Maximize expected return minus risk aversion times variance
portfolio_return = mu_bar.values * w
portfolio_variance = cp.quad_form(
    w, cov_matrix, assume_PSD=True
)
# portfolio_variance = np.dot(w.T, np.dot(cov_matrix, w))
utility = (
    0.5 * portfolio_variance - (1/delta) * portfolio_return
)

In [234]:
delta

0.39789740238532967

In [ ]:
objective = cp.Minimize(utility)

# Solve the problem
try:
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS)

    if prob.status in ["optimal", "optimal_inaccurate"]:
        weights = w.value
        weights = pd.Series(weights, index=assets)
        result["weights"] = weights.values
        result["status"] = "success"
    else:
        result["status"] = "failure"

except Exception as e:
    result["status"] = "failure"



In [273]:
print(weights)
print(np.max(weights))
print(np.min(weights))
print(sum(weights))
print(sum(abs(weights)))

[-2.17017348e-08 -9.18781559e-08 -4.91602835e-08 ... -1.08758926e-07
 -2.29443830e-08 -2.71567929e-08]
1.0000001090543684
-1.0000002377419284
0.999999998328761
5.000138947946764


In [274]:
portfolio_return_opti = np.sum(returns.mean().values * weights)
portfolio_variance_opti = weights.T @ cov_matrix @ weights
portfolio_vol_opti = np.sqrt(portfolio_variance_opti)

print(portfolio_return_opti)
print(portfolio_vol_opti)

0.06059821322121162
1.6367607663578616


In [275]:
tracking_error_optimized = np.sqrt((weights-x0) @ cov_matrix @ (weights-x0))
tracking_error_optimized

1.5132030038452366

In [338]:
def compute_mu_bar(tau):

    # Calculate omega (uncertainty matrix)
    omega = np.diag(np.diag(tau * P @ cov_matrix.values @ P.T))

    # Set initial weights x0 (assumed market cap weights or equal weights)
    x0 = np.array([1/num_assets] * num_assets)

    # Compute initial portfolio return and volatility
    rf = 0.0  # Risk-free rate, adjust if necessary
    initial_returns = returns.mean()
    initial_portfolio_return = np.dot(x0, initial_returns)
    initial_portfolio_vol = np.sqrt(np.dot(x0.T, np.dot(cov_matrix, x0)))
    initial_portfolio_sharpe = (initial_portfolio_return - rf) / initial_portfolio_vol

    # Compute implied risk aversion parameter (delta)
    delta = initial_portfolio_sharpe / initial_portfolio_vol

    # Compute the implied equilibrium returns (pi)
    pi = delta * cov_matrix.dot(x0)

    gamma = tau * cov_matrix
    gamma

    M = P.dot(gamma).dot(P.T) + omega

    # Regularize covariance matrix (after Ledoit-Wolf shrinkage)
    epsilon = 1e-8  # Small regularization factor
    M += epsilon * np.eye(M.shape[0])

    M_inverse = np.linalg.inv(M)

    mu_bar = pi + gamma.dot(P.T).dot(M_inverse).dot(Q - P.dot(pi))

    return mu_bar

def get_tracking_error(tau):

    mu_bar = compute_mu_bar(tau)
    if mu_bar is not None:
        print("mu_bar isn't None")
    else:
        print("mu_bar is none")
        return np.inf

    assets = len(returns)

    # Prepare a result dictionary
    result = {
        "weights": None,
        "mean_returns": None,
        "cov_matrix": cov_matrix,
        "status": None,
    }

    # Define variables
    w = cp.Variable(num_assets)

    # Objective function
    portfolio_variance = cp.quad_form(w, cov_matrix, assume_PSD=True)

    # Initialize constraints list
    constraints = []


    constraints.append(w >= -1)
    constraints.append(w <= 1)
    constraints.append(cp.sum(w) == 1)

    # Maximize Utility Function: Maximize expected return minus risk aversion times variance
    portfolio_return = mu_bar.values * w
    portfolio_variance = cp.quad_form(
        w, cov_matrix, assume_PSD=True
    )
    # portfolio_variance = np.dot(w.T, np.dot(cov_matrix, w))
    utility = (
        0.5 * portfolio_variance - (1/delta) * portfolio_return
    )

    objective = cp.Minimize(utility)

    # Solve the problem
    try:
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS)

        if prob.status in ["optimal", "optimal_inaccurate"]:
            weights = w.value
            weights = pd.Series(weights, index=assets)
            tracking_error = np.sqrt((weights - x0).T @ cov_matrix @ (weights - x0))
            result["status"] = "success"
        else:
            result["status"] = "failure"
            tracking_error = np.inf

    except Exception as e:
        result["status"] = "failure"
        tracking_error = np.inf

    return tracking_error



In [339]:
# Function to find the difference between calculated and target tracking error
def tracking_error_difference(tau, target_tracking_error):
    tracking_error = get_tracking_error(tau)
    return tracking_error - target_tracking_error

In [340]:
# Function to find a valid interval where the function changes sign
def find_valid_interval(func, target, initial_upper=10, max_upper=1e6):
    tau_lower = 1e-6
    tau_upper = initial_upper
    f_lower = func(tau_lower, target)
    f_upper = func(tau_upper, target)

    while f_lower * f_upper > 0 and tau_upper < max_upper:
        tau_upper *= 10
        f_upper = func(tau_upper, target)
        print(f"Trying tau_upper = {tau_upper}, function value = {f_upper}")

    if f_lower * f_upper < 0:
        return tau_lower, tau_upper
    else:
        return None, None  # Could not find a valid interval


In [345]:
from scipy.optimize import brentq


# Root-finding to optimize tau
# Target tracking error
target_tracking_error = 0.05  # Example target tracking error

# Find valid interval
tau_lower, tau_upper = find_valid_interval(tracking_error_difference, target_tracking_error)


if tau_lower is not None:
    print(f"Found valid interval: [{tau_lower}, {tau_upper}]")
    try:
        tau_optimal = brentq(
            tracking_error_difference, tau_lower, tau_upper, args=(target_tracking_error,)
        )
        print(f"Optimal tau: {tau_optimal}")
    except ValueError as e:
        print(f"Root finding failed: {e}")
        tau_optimal = None
else:
    print("Could not find a valid interval where the function changes sign.")
    tau_optimal = None

# After finding the optimal tau, compute the optimal weights
if tau_optimal is not None:
    mu_bar_optimal = compute_mu_bar(tau_optimal)
    if mu_bar_optimal is not None:
        # Re-run the optimization with the optimal tau
        w = cp.Variable(num_assets)
        portfolio_variance = cp.quad_form(w, cov_matrix)
        portfolio_return = mu_bar_optimal @ w
        utility = 0.5 * portfolio_variance - (1/delta) * portfolio_return
        objective = cp.Minimize(utility)
        constraints = [
            w >= -1,
            w <= 1,
            cp.sum(w) == 1,
        ]
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS)
        if prob.status in ["optimal", "optimal_inaccurate"]:
            optimal_weights = w.value
            print("Optimal weights:")
            print(optimal_weights)
        else:
            print("Optimization failed with the optimal tau.")
    else:
        print("Failed to compute mu_bar with the optimal tau.")
else:
    print("Unable to find optimal tau. Please adjust the interval or check the function.")

mu_bar isn't None
mu_bar isn't None
mu_bar isn't None
Trying tau_upper = 100, function value = inf
mu_bar isn't None
Trying tau_upper = 1000, function value = inf
mu_bar isn't None
Trying tau_upper = 10000, function value = inf
mu_bar isn't None
Trying tau_upper = 100000, function value = inf
mu_bar isn't None
Trying tau_upper = 1000000, function value = inf
Could not find a valid interval where the function changes sign.
Unable to find optimal tau. Please adjust the interval or check the function.


In [326]:
import cvxpy as cp
import numpy as np
import pandas as pd
from scipy.optimize import brentq
from scipy.linalg import solve

# Assuming all necessary data is already defined:
# - returns
# - cov_matrix (prior covariance matrix)
# - x0 (initial or benchmark weights)
# - delta (risk aversion coefficient)
# - pi (market equilibrium returns)
# - P, Q, Omega (views and uncertainties)
# - target_tracking_error (your desired tracking error)
# - num_assets (number of assets)

# Function to compute mu_bar given tau
def compute_mu_bar(tau):
    Sigma = cov_matrix
    tau_Sigma_inv = np.linalg.inv(tau * Sigma)
    Omega_inv = np.linalg.inv(omega)
    M = tau_Sigma_inv + P.T @ Omega_inv @ P
    M_pinv = pinv(M)
    # # Regularize covariance matrix (after Ledoit-Wolf shrinkage)
    # epsilon = 1e-6  # Small regularization factor
    # # Regularize M to ensure it's invertible
    # M += epsilon * np.eye(M.shape[0])

    right_hand_side = tau_Sigma_inv @ pi + P.T @ Omega_inv @ Q

    mu_bar = M_pinv @ right_hand_side

    # # Solve the linear system M * mu_bar = right_hand_side
    # try:
    #     mu_bar = solve(M, right_hand_side, assume_a='pos')
    # except np.linalg.LinAlgError as e:
    #     print(f"Linear system solve failed: {e}")
    #     mu_bar = None
    return mu_bar

# Function to compute tracking error for a given tau
def get_tracking_error(tau):
    mu_bar = compute_mu_bar(tau)
    if mu_bar is None:
        return np.inf  # Return a large value if mu_bar couldn't be computed
    else:
        print("mu_bar isn't none")

    # Define variables
    w = cp.Variable(num_assets)
    
    # Objective function
    portfolio_variance = cp.quad_form(w, cov_matrix)
    portfolio_return = mu_bar @ w
    
    utility = 0.5 * portfolio_variance - (1/delta) * portfolio_return
    objective = cp.Minimize(utility)
    
    # Constraints
    constraints = [
        w >= -1,
        w <= 1,
        cp.sum(w) == 1,
        cp.norm1(w) <= 5,
    ]
    
    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS)
    
    if prob.status in ["optimal", "optimal_inaccurate"]:
        weights = w.value
        tracking_error = np.sqrt((weights - x0).T @ cov_matrix @ (weights - x0))
        return tracking_error
    else:
        return np.inf  # Return a large value if optimization fails
    
# Function to find the difference between calculated and target tracking error
def tracking_error_difference(tau, target_tracking_error):
    tracking_error = get_tracking_error(tau)
    return tracking_error - target_tracking_error

# Root-finding to optimize tau
tau_lower = 1e-6
tau_upper = 10
target_tracking_error = 0.05  # Example target tracking error

try:
    tau_optimal = brentq(
        tracking_error_difference, tau_lower, tau_upper, args=(target_tracking_error,)
    )
    print(f"Optimal tau: {tau_optimal}")
except ValueError as e:
    print(f"Root finding failed: {e}")
    # You may need to adjust tau_lower and tau_upper

# After finding the optimal tau, compute the optimal weights
mu_bar_optimal = compute_mu_bar(tau_optimal)

if mu_bar_optimal is not None:
    # Re-run the optimization with the optimal tau
    w = cp.Variable(num_assets)
    portfolio_variance = cp.quad_form(w, cov_matrix)
    portfolio_return = mu_bar_optimal @ w
    utility = 0.5 * portfolio_variance - (1/delta) * portfolio_return
    objective = cp.Minimize(utility)
    constraints = [
        w >= -1,
        w <= 1,
        cp.sum(w) == 1,
        cp.norm1(w) <= 5,
    ]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS)
    if prob.status in ["optimal", "optimal_inaccurate"]:
        optimal_weights = w.value
        print("Optimal weights:")
        print(optimal_weights)
    else:
        print("Optimization failed with the optimal tau.")
else:
    print("Failed to compute mu_bar with the optimal tau.")

Root finding failed: Singular matrix


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'